In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from datetime import timedelta
import scipy.optimize as opt

# Users just change this part of the code:
# num_days = how many days from today do you want to predict 
# prediction_file_name = csv file that will be created with just states, counties, prediction
# jhu_and_prediction_file_name = csv file that will be created with entire jhu data and predictions appended on the end

In [4]:
num_days = 10
prediction_file_name = 'v2_pred_10days_from05222020.csv'
jhu_and_prediciton_file_name = 'v2_full_pred_10days_from05222020.csv'

In [17]:
jhu_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
jhu = pd.read_csv(jhu_url)
first_case_day = jhu.replace(0, np.nan).iloc[:, 11:].apply(lambda series: series.first_valid_index(), axis = 1)
first_day = pd.to_datetime('1/22/20')
def conv_delta_to_int (dt):
    return int(str(dt).split(" ")[0])
conv_to_int = np.vectorize(conv_delta_to_int)
first_cases = pd.to_datetime(first_case_day)
first_cases= first_cases.fillna(pd.datetime.now())
first_cases = conv_to_int(pd.to_datetime(first_cases) - first_day)
jhu.insert(2, 'first_case_day', first_cases)

numeric_cols = [col for col in jhu if jhu[col].dtype.kind != 'O']
cols = numeric_cols[5:]
jhu[cols] += 1

C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if __name__ == '__main__':


In [6]:
decline_day = 75  # what day should we start looking for a decline
days_of_decline = 7 # how many days of decline in the range (decline day to today) will trigger a model that is declining
val_set_size = 1

epochs = 5 # how many times does scipy.optimize run each time - only local maximum so running multiple times and taking best
first_day_column = jhu.columns.get_loc("1/22/20")
def conv_delta_to_int (dt):
    return int(str(dt).split(" ")[0])
conv_to_int = np.vectorize(conv_delta_to_int)

t = conv_to_int(pd.to_datetime(jhu.columns[first_day_column:]) - first_day) # current time stamps
t_ = np.arange(250) # time stamps with further range, used for predicting

In [7]:
def exponential(t, a,b):
    return a*(np.exp(b*t))

def logistic(t, c, a,b):
    return c / (1. + a*np.exp(-1.0*b * t))

def rectified(t,m,b, first_day):
    def one_rect(t,m,b, first):
        if m*t + b<1 and t<first_day:
            return 1
        else:
            return m*t+b
    rect = np.vectorize(one_rect)
    return rect(t,m,b,first_day)
    

def before_sign(x):
    if x <0:
        return -1
    if x>0:
        return 1
    else:
        return 0
def after_sign(x):
    if x >0:
        return -1
    if x<0:
        return 1
    else:
        return 0

before_vect = np.vectorize(before_sign)
after_vect = np.vectorize(after_sign)

def get_all_inflections(vals, window_size = 25, base_jump = 5, peak_decline = 10):
    peak_inf = 0
    max_peak = 0
    case_der = np.diff(vals,2)
    base_test = np.absolute(case_der) > base_jump
    if len([i for i, x in enumerate(base_test) if x]) ==0:
        return None, None
    else:
        base_inf = [i for i, x in enumerate(base_test) if x][0]
    test = case_der <-peak_decline
    test2 = np.array([i for i, x in enumerate(test) if x]) >= decline_day
    if len([i for i, x in enumerate(test2) if x]) < days_of_decline:
        return None, base_inf
    for idx, val in enumerate(case_der):
        if idx ==0:
            continue
        if idx == len(case_der)-1:
            continue
        if idx <window_size:
            if np.sum(before_vect(case_der[0:idx])) + np.sum(after_vect(case_der[idx+1:idx+1+window_size]))> max_peak:
                peak_inf = idx
                max_peak = np.sum(before_vect(case_der[0:idx])) + np.sum(after_vect(case_der[idx+1:idx+1+window_size]))
        if idx >= window_size and idx < len(case_der)-window_size:
            if np.sum(before_vect(case_der[idx-window_size:idx])) + np.sum(after_vect(case_der[idx+1:idx+1+window_size]))> max_peak:
                peak_inf = idx
                max_peak = np.sum(before_vect(case_der[idx-window_size:idx])) + np.sum(after_vect(case_der[idx+1:idx+1+window_size]))
        if idx >=window_size and idx >= len(case_der)-window_size:
            if np.sum(before_vect(case_der[idx-window_size:idx])) + np.sum(after_vect(case_der[idx+1:]))> max_peak:
                peak_inf = idx
                max_peak = np.sum(before_vect(case_der[idx-window_size:idx])) + np.sum(after_vect(case_der[idx+1:]))
    return peak_inf, base_inf

peak_dist = []
for i, row in jhu.iterrows():
    p, b = get_all_inflections(row[first_day_column:])
    try:
        peak_dist.append(np.absolute(p-b))
    except:
        continue
avg_peak = int(np.round(np.average(peak_dist)))

In [8]:
def pred_county(county_row, new_days = num_days, current_day = t[-1], val_size = val_set_size):
    #print(county_row)
    first = county_row['first_case_day']
    peak, base = get_all_inflections(county_row[first_day_column:])
    true_vals = np.array(county_row[first_day_column:].astype(int))
    train_set = true_vals[:-val_size]
    valid_set = true_vals[-val_size:]
    
    if base == None:
        #print('rectified')
        p0 = (1,-40)
        bounds = ([0,-1500], [10,0])
        f_rectified = lambda t_,a,b: rectified(t_,a,b,first)
        (m,p),_ = opt.curve_fit(f_rectified,t[:-val_size],true_vals[:-val_size], bounds= bounds, p0 = p0)
        predictions = [rectified(t,m,p,first) for t in t_[current_day+1:current_day+new_days+1]]
        predictions = np.array(predictions).reshape(1,-1)[0]
        pred_time = t_[current_day:current_day+new_days+1]
        train_error = np.sum(np.square(np.log(train_set) - np.log([rectified(t,m,p,first) for t in t[:-val_size]])))
        val_error = np.sum(np.square(np.log(valid_set) - np.log([rectified(t,m,p,first) for t in t[-val_size:]])))
        if val_error>5:
            predictions = np.full(len(predictions), train_set[-1])
            val_error = np.sum(np.square(np.log(valid_set) - np.log(train_set[-1])))
        
        return predictions, pred_time, val_error
        
    if peak == None:
        peak = base + avg_peak
        if peak < current_day:
            peak = current_day-1 
    if current_day >= peak:
        #print('logistic')
        
        final_c,final_a,final_b = 0,0,0
        val_error_test = 1000000
        for j in np.arange(epochs):
            #p0 = (1000, 100, 0.3)
            bounds = ([0,0,0], [10000000, 100000000, 3])
            (c,a,b),_ = opt.curve_fit(logistic,t[:-val_size],true_vals[:-val_size], bounds = bounds)
            train_error = np.sum(np.square(np.log(true_vals[np.nonzero(train_set)]) - np.log([logistic(t,c,a,b) for t in t[np.nonzero(train_set)]])))
            val_error = np.sum(np.square(np.log(valid_set) - np.log([logistic(t,c,a,b) for t in t[-val_size:]])))
            if val_error < val_error_test:
                final_c, final_a,final_b = c,a,b
                val_error_test = val_error
        predictions = [logistic(t,final_c,final_a,final_b) for t in t_[current_day+1:current_day+new_days+1]]
        pred_time = t_[current_day+1:current_day+new_days+1]
    elif current_day< peak: #and current_day  + new_days <= peak:
        #print('exponential')
        final_a,final_b = 0,0
        val_error_test = 1000000
        for j in np.arange(epochs):
            p0 = (1,.5)
            bounds = ([0.01,0], [10,1])
            (a,b),_ = opt.curve_fit(exponential,t[:-val_size],true_vals[:-val_size], bounds = bounds, p0=p0)
            train_error = np.sum(np.square(np.log(true_vals[np.nonzero(train_set)]) - np.log([exponential(t,a,b) for t in t[np.nonzero(train_set)]])))
            val_error = np.sum(np.square(np.log(valid_set) - np.log([exponential(t,a,b) for t in t[-val_size:]])))
            if val_error < val_error_test:
                final_a, final_b = a,b
                val_error_test = val_error
        predictions = [exponential(t,final_a,final_b) for t in t_[current_day+1:current_day+new_days+1]]
        pred_time = t_[current_day+1:current_day+new_days+1]
    '''elif current_day< peak and current_day  + new_days >= peak:
        # piecewise prediction
        print('piecewise')
        final__a,final__b = 0,0
        val_error_test = 1000000
        for j in np.arange(epochs):
            p0 = (1,.5)
            bounds = ([0.01,0], [10,1])
            (a,b),_ = opt.curve_fit(exponential,t[:-val_size],true_vals[:-val_size], bounds = bounds, p0 =p0)
            error = np.sum(np.square(np.log(true_vals[np.nonzero(true_vals)]) - np.log([exponential(t,a,b) for t in t[np.nonzero(true_vals)]])))
            #val_error = np.sum(np.square(np.log(valid_set) - np.log([exponential(t,a,b) for t in t[-val_size:]])))
            if error < val_error_test:
                final__a, final__b = a,b
                val_error_test = error
        
        predictions = [int(np.round(exponential(t,final__a,final__b))) for t in t_[current_day+1:peak+1]]
        pred_time = t_[current_day+1:peak+1]
        
        true_pred_vals = np.concatenate((true_vals,predictions), axis=None)
        final_c,final_a,final_b = 0,0,0
        error_test = 100000
        for j in np.arange(epochs):
            #p0 = (1000, 100, 0.3)
            bounds = ([0,0,0], [10000000, 1000000, 1])
            
            (c,a,b),_ = opt.curve_fit(logistic,t_[:len(true_pred_vals)],true_pred_vals, bounds = bounds)
            error =  np.sum(np.square(np.log(true_pred_vals[np.nonzero(true_pred_vals)]) - np.log([logistic(t,c,a,b) for t in t_[np.nonzero(true_pred_vals)]])))
            if error < error_test:
                final_c, final_a,final_b = c,a,b
                error_test = error
        
        predictions = np.concatenate((predictions,[logistic(t,final_c,final_a,final_b) for t in t_[peak+1:current_day+new_days+1]]), axis=None)
        pred_time = np.concatenate((pred_time, t_[peak+1:current_day+new_days+1]), axis = None)'''
        
    return predictions, pred_time, val_error_test

In [9]:
def predict(df = jhu, current_day = t[-1], new_days = num_days):
    prediction_array = t_[current_day+1:current_day+new_days+1]
    prediction_df = pd.DataFrame()
    prediction_df['State'] = df['Province_State']
    prediction_df['County'] = df['Admin2']
    val_errors = []
    for i, row in jhu.iterrows():
        try:
            predictions, time, val_error = pred_county(row)
            prediction_array = np.vstack((prediction_array, predictions))
            val_errors.append(val_error)
                
        except:
            print('error', i)
        
    def conv_int_to_delta(day):
        return first_day + timedelta(days = int(day))
    conv_to_delta = np.vectorize(conv_int_to_delta)
    pred_df = pd.DataFrame(data = prediction_array[1:,:], columns = conv_to_delta(prediction_array[0,:]))
    predictions_df = pd.concat([prediction_df, pred_df], axis=1)
    full_df = pd.concat([jhu,pred_df], axis = 1)
    
    return predictions_df,full_df, val_errors


In [10]:
predictions, full_df, validation_errors = predict()
print('Validation error average: ', np.average(validation_errors ))
predictions.to_csv(prediction_file_name)
full_df.to_csv(jhu_and_prediciton_file_name)

C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in log


error 1380
Validation error average:  0.5115027456686246
